# Constraining Stress Magnitudes from Wellbore Failure

This notebook demonstrates how to constrain stress magnitudes using wellbore failure observations with GeoSuite.

## Overview

Wellbore failure (breakouts and tensile fractures) provides critical constraints on in-situ stress magnitudes:
- **Breakouts**: Indicate where SHmax exceeds failure threshold
- **Tensile fractures**: Indicate where Shmin is too low
- **Stress polygon**: Combines failure observations with LOT data

GeoSuite provides functions for:
- Stress polygon analysis
- Determining stress regime from principal stresses
- Visualizing stress constraints

This notebook will show you how to:

1. Load stress and failure data
2. Calculate stress constraints from failure observations
3. Visualize stress polygons and constraints

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs
from geosuite.geomech import (
    calculate_overburden_stress,
    calculate_hydrostatic_pressure,
    stress_polygon_limits,
    plot_stress_polygon,
    determine_stress_regime
)

print("GeoSuite imported successfully!")

## 1. Load Stress Data

Load well log data and calculate basic stress profile.

In [ ]:
# Load demo well log data
df = load_demo_well_logs()

# Find depth and density columns
depth_col = 'depth_m' if 'depth_m' in df.columns else 'DEPTH'
density_cols = [col for col in df.columns if 'RHOB' in col.upper() or 'DENSITY' in col.upper()]

depth = df[depth_col].values
rhob = df[density_cols[0]].values if density_cols else np.ones(len(depth)) * 2.5

# Calculate overburden stress
sv = calculate_overburden_stress(depth, rhob)
ph = calculate_hydrostatic_pressure(depth, rho_water=1.03)

print(f"Calculated stress profile for {len(depth)} depths")
print(f"Depth range: {depth.min():.1f} to {depth.max():.1f} m")
print(f"Sv range: {sv.min():.1f} to {sv.max():.1f} MPa")

## 2. Define Wellbore Failure Observations

In practice, you would load actual breakout and tensile fracture data from image logs or caliper data.

In [ ]:
# Example wellbore failure observations
# Breakouts indicate SHmax is too high (exceeds failure threshold)
# Tensile fractures indicate Shmin is too low

failure_data = {
    'depth': [1500, 2000, 2500],  # Depths with failure observations
    'breakout_width': [30, 45, 20],  # Breakout width in degrees
    'tensile_fractures': [False, True, False]  # Presence of tensile fractures
}

print("Wellbore Failure Observations:")
for i, d in enumerate(failure_data['depth']):
    idx = np.argmin(np.abs(depth - d))
    sv_val = sv[idx]
    pp_val = ph[idx]
    
    print(f"\nDepth: {d:.0f} m")
    print(f"  Sv: {sv_val:.1f} MPa, Pp: {pp_val:.1f} MPa")
    print(f"  Breakout width: {failure_data['breakout_width'][i]}°")
    print(f"  Tensile fractures: {failure_data['tensile_fractures'][i]}")

## 3. Calculate Stress Constraints

Use stress polygon limits to determine allowable SHmax ranges at failure depths.

In [ ]:
# Calculate stress constraints at failure depths
constraints = []

for i, d in enumerate(failure_data['depth']):
    idx = np.argmin(np.abs(depth - d))
    sv_val = sv[idx]
    pp_val = ph[idx]
    
    # Estimate Shmin (in practice, use LOT data or other constraints)
    # For this example, assume Shmin = 0.7 * Sv (typical for normal faulting)
    shmin_est = 0.7 * sv_val
    
    # Calculate stress polygon limits
    limits = stress_polygon_limits(
        sv=sv_val,
        pp=pp_val,
        shmin=shmin_est,
        mu=0.6,
        cohesion=0.0
    )
    
    # Breakouts indicate SHmax is near or above failure threshold
    # For normal faulting, SHmax should be in normal faulting range
    if failure_data['breakout_width'][i] > 0:
        # Breakout width > 0 means SHmax is high enough to cause failure
        shmax_min = limits['normal'][0]  # Minimum for normal faulting
        shmax_max = limits['normal'][1]  # Maximum (Sv)
        constraint_type = 'Breakout (SHmax constrained)'
    else:
        shmax_min = None
        shmax_max = None
        constraint_type = 'No constraint'
    
    constraints.append({
        'depth': d,
        'sv': sv_val,
        'pp': pp_val,
        'shmin_est': shmin_est,
        'shmax_min': shmax_min,
        'shmax_max': shmax_max,
        'constraint_type': constraint_type
    })

# Display constraints
constraints_df = pd.DataFrame(constraints)
print("\nStress Constraints from Wellbore Failure:")
print(constraints_df.to_string(index=False))

## 4. Visualize Stress Polygon with Constraints

Plot stress polygon showing allowable stress regimes and failure constraints.

In [ ]:
# Create stress polygon plot
# Use estimated Shmin profile
shmin_profile = 0.7 * sv  # Simplified estimate

fig = plot_stress_polygon(
    depths=depth,
    sv=sv,
    pp=ph,
    shmin=shmin_profile,
    mu=0.6,
    cohesion=0.0,
    title='Stress Polygon with Wellbore Failure Constraints'
)

# Mark failure depths
ax = fig.axes[0]
for d in failure_data['depth']:
    ax.axhline(y=d, color='red', linestyle=':', linewidth=2, alpha=0.7, label='Failure Depth')

plt.show()

## 5. Determine Stress Regime

Use `determine_stress_regime()` to classify the stress regime at each depth.

In [ ]:
# Determine stress regime at failure depths

print("Stress Regime Analysis:")
for i, d in enumerate(failure_data['depth']):
    idx = np.argmin(np.abs(depth - d))
    sv_val = sv[idx]
    shmin_val = shmin_profile[idx]
    
    # Estimate SHmax (use middle of normal faulting range)
    limits = stress_polygon_limits(sv_val, ph[idx], shmin_val, mu=0.6)
    shmax_est = (limits['normal'][0] + limits['normal'][1]) / 2
    
    regime = determine_stress_regime(sv_val, shmin_val, shmax_est)
    
    print(f"\nDepth: {d:.0f} m")
    print(f"  Sv: {sv_val:.1f} MPa")
    print(f"  Shmin: {shmin_val:.1f} MPa")
    print(f"  SHmax: {shmax_est:.1f} MPa")
    print(f"  Regime: {regime.replace('_', ' ').title()}")

## 6. Summary

This notebook demonstrated:

-  Calculating stress profiles from well logs
-  Using wellbore failure observations to constrain stresses
-  Stress polygon analysis with `stress_polygon_limits()`
-  Visualizing stress constraints with `plot_stress_polygon()`
-  Determining stress regime with `determine_stress_regime()`

### Key Functions Used

- `calculate_overburden_stress()`: Compute Sv from density log
- `stress_polygon_limits()`: Determine SHmax constraints
- `plot_stress_polygon()`: Visualize stress regimes
- `determine_stress_regime()`: Classify faulting regime

### Next Steps

- Load actual breakout data from image logs or caliper data
- Combine with LOT data for better Shmin constraints
- Use multiple failure observations to refine stress model
- Calculate mud weight windows for drilling operations